In [26]:
'''PIPELINE for mAP Calculation'''

'PIPELINE for mAP Calculation'

In [27]:
'''Step 1. Generate predicted JSON files from Apurba OCR system'''

'Step 1. Generate predicted JSON files from Apurba OCR system'

In [8]:
'''Step 2. Rename all the predicted JSON files to its corresponding image filename as Apurba OCR changes the JSON filename'''

'2. Rename all the predicted JSON files to its corresponding image filename as Apurba OCR changes the JSON filename'

In [18]:
import os
import shutil

# set the paths to the directories containing the image and JSON files
image_dir = "/kaggle/input/1-generated-prediction-json/images/images"
json_dir = "/kaggle/input/1-generated-prediction-json/corresponding-json/corresponding-json"
output_dir = "/kaggle/working/"

# create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [19]:
# get the list of image filenames
image_filenames = os.listdir(image_dir)

In [21]:
print("Image files in this directory: \n\n", image_filenames)

Image files in this directory: 

 ['00b6fc4e-0263-456a-a7a8-d183d4877c49.png', '0004b1c3-6260-49f1-bc82-6e6ae94b98f0.png', '003ba068-116f-461d-9d70-204204e3b61a.png', '025c9854-c6dd-442b-be91-33434fd3f603.png']


In [24]:
# iterate over the JSON files
for json_filename in os.listdir(json_dir):
    # get the first 8 characters of the JSON filename
    json_prefix = json_filename[:8]
    print("1st 8 characters of this JSON file: \n\n", json_prefix)
    
    # iterate over the image filenames
    for image_filename in image_filenames:
        # get the first 8 characters of the image filename
        image_prefix = image_filename[:8]
        print("1st 8 characters of this IMAGE file: \n\n", image_prefix)

        # check if the JSON filename matches the image filename
        if json_prefix == image_prefix:
            # rename the JSON file with the corresponding image filename
            # remove the ".png" extension from the image filename and replace it with ".json"
            new_json_filename = image_filename[:-4] + ".json" 
#             os.rename(os.path.join(json_dir, json_filename), os.path.join(output_dir, new_json_filename))
            shutil.copy(os.path.join(json_dir, json_filename), os.path.join(output_dir, new_json_filename))
#             os.remove(os.path.join(json_dir, json_filename))

            # print a message indicating that the file was renamed
            print(f"Renamed {json_filename} to {new_json_filename}")
            break
    else:
        # if no match was found, copy the JSON file to the output directory without renaming it
        shutil.copy(os.path.join(json_dir, json_filename), output_dir)

1st 8 characters of this JSON file: 

 00b6fc4e
1st 8 characters of this IMAGE file: 

 00b6fc4e
Renamed 00b6fc4e-0263-456a-a7a8-d194caa74d9e.json to 00b6fc4e-0263-456a-a7a8-d183d4877c49.json
1st 8 characters of this JSON file: 

 003ba068
1st 8 characters of this IMAGE file: 

 00b6fc4e
1st 8 characters of this IMAGE file: 

 0004b1c3
1st 8 characters of this IMAGE file: 

 003ba068
Renamed 003ba068-116f-461d-9d70-20665028d250.json to 003ba068-116f-461d-9d70-204204e3b61a.json
1st 8 characters of this JSON file: 

 025c9854
1st 8 characters of this IMAGE file: 

 00b6fc4e
1st 8 characters of this IMAGE file: 

 0004b1c3
1st 8 characters of this IMAGE file: 

 003ba068
1st 8 characters of this IMAGE file: 

 025c9854
Renamed 025c9854-c6dd-442b-be91-33ca259230c4.json to 025c9854-c6dd-442b-be91-33434fd3f603.json
1st 8 characters of this JSON file: 

 0004b1c3
1st 8 characters of this IMAGE file: 

 00b6fc4e
1st 8 characters of this IMAGE file: 

 0004b1c3
Renamed 0004b1c3-6260-49f1-bc82-6

In [28]:
'''Step 3. Combine all the JSON files in 1 single JSON in COCO format as you will get 1 single predicted JSON for 1 image document and in another format'''

'Step 3. Combine all the JSON files in 1 single JSON in COCO format as you will get 1 single predicted JSON for 1 image document and in another format'

In [29]:
import json

# Define the directory where the JSON files are located
directory = '/kaggle/input/2-renamed-predicted-json/2-renamed-predicted-json/'

# Create a new dictionary in COCO format
coco_dict = {
    "info": {
        "year": "2023",
        "version": "1",
        "description": "Converted to COCO json format using custom python script",
        "contributor": "Md. Mutasim Billah Abu Noman Akanda",
        "date_created": "2023/04/05"
    },
    "licenses": [],
    "categories": [
        {
            "id": 0,
            "name": "TextBlock"
        },
        {
            "id": 1,
            "name": "paragraph"
        },
        {
            "id": 2,
            "name": "TableRegion"
        }
    ],
    "images": [],
    "annotations": [],
}

# Iterate through each JSON file in the directory
image_id = 0
segment_id = 0
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        # Load the JSON file
        with open(os.path.join(directory, filename), 'r') as f:
            data = json.load(f)

        # Get image information and add to the COCO dictionary
        width = data['width']
        height = data['height']
        coco_dict['images'].append({
            "id": image_id,
            "file_name": filename.replace('.json', '.png'),
            "width": width,
            "height": height
        })

        # Iterate through the segments and add to the COCO dictionary
        for segment_key, segment_value in data['segments'].items():
            # Check if the category exists in the COCO dictionary
            category_id = None
            for cat in coco_dict['categories']:
                if cat['name'] == segment_value['type']:
                    category_id = cat['id']
                    break
            # If the category does not exist, add a new category to the COCO dictionary
            if category_id is None:
                category_id = len(coco_dict['categories'])
                coco_dict['categories'].append({
                    "id": category_id,
                    "name": segment_value['type']
                })

            bbox = [
                segment_value['points'][0]['x'],
                segment_value['points'][0]['y'],
                segment_value['points'][2]['x'],
                segment_value['points'][1]['y']
            ]

            width = segment_value['points'][2]['x'] - segment_value['points'][0]['x']
            height = segment_value['points'][1]['y'] - segment_value['points'][0]['y']
            area = width * height

            coco_dict['annotations'].append({
                "id": segment_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": bbox,
                "width" : width,
                "height" : height,
                "area": area,
                "iscrowd": 0
            })

            segment_id += 1

        image_id += 1

# Save the COCO dictionary as a JSON file
with open('converted_single-coco_format.json', 'w') as f:
    json.dump(coco_dict, f)

In [30]:
'''Step 4. Map between the ground truth and prediction JSON as category id, category name are different'''

'Step 4. Map between the ground truth and prediction JSON as category id, category name are different'

In [31]:
# Load ground truth JSON file
with open('/kaggle/input/0-badlad-groundtruth/0-badlad-groundtruth/BDLAD_train_coco.json') as f:
    ground_truth = json.load(f)

# Load prediction JSON file
with open('/kaggle/input/3-converted-single-coco-format/3-converted-single-coco-format/converted_single-coco_format.json') as f:
    prediction = json.load(f)

In [33]:
# Extract the category lists from both files
gt_categories = ground_truth['categories']
pred_categories = prediction['categories']

# Print the number and names of categories in both files
print(f"Number of categories in ground truth: {len(gt_categories)}")
print("Ground truth categories:")
for cat in gt_categories:
    print(f"  - {cat['name']}")

print(f"\nNumber of categories in predicted file: {len(pred_categories)}")
print("Predicted categories:")
for cat in pred_categories:
    print(f"  - {cat['name']}")

Number of categories in ground truth: 4
Ground truth categories:
  - paragraph
  - text_box
  - image
  - table

Number of categories in predicted file: 3
Predicted categories:
  - TextBlock
  - paragraph
  - TableRegion


In [34]:
# Ground truth object
print("Number of Image IDs in Ground Truth:", len(ground_truth["images"]))
print("Number of Annotations in Ground Truth:", len(ground_truth["annotations"]))

# Predicted object
print("Number of Image IDs in Predicted:", len(prediction["images"]))
print("Number of Annotations in Predicted:", len(prediction["annotations"]))

Number of Image IDs in Ground Truth: 20367
Number of Annotations in Ground Truth: 425109
Number of Image IDs in Predicted: 4
Number of Annotations in Predicted: 910


In [35]:
# Modify the categories and category IDs
for category in prediction['categories']:
    if category['name'] == 'TextBlock':
        category['name'] = 'text_box'
        category['id'] = 1
    elif category['name'] == 'paragraph':
        category['name'] = 'paragraph'
        category['id'] = 0
    elif category['name'] == 'TableRegion':
        category['name'] = 'table'
        category['id'] = 3
    # Add a new category 'image'
    elif category['name'] == 'image':
        category['id'] = 2

In [37]:
# Modify category_id in annotations dictionary
for annotation in prediction['annotations']:
    category_id = annotation['category_id']
    if category_id == 0:
        annotation['category_id'] = 1
    elif category_id == 1:
        annotation['category_id'] = 0
    elif category_id == 2:
        annotation['category_id'] = 3

In [38]:
# Save the updated prediction JSON file
with open('mapped_prediction.json', 'w') as f:
    json.dump(prediction, f, indent=4)

In [40]:
'''Step 5. Calculate mAP comparing ground truth and predictions'''

'Step 5. Calculate mAP comparing ground truth and predictions'

In [41]:
# writing method for loading json file
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [44]:
# loading ground truth JSON
gt_file_path = '/kaggle/input/0-badlad-groundtruth/0-badlad-groundtruth/BDLAD_train_coco.json'
ground_truth = load_json(gt_file_path)

# loading mapped prediction JSON
pred_file_path = '/kaggle/input/4-mapped-prediction-json/4-mapped-prediction-json/mapped_prediction.json'
predictions = load_json(pred_file_path)

In [45]:
# Extract the category lists from both files
gt_categories = ground_truth['categories']
pred_categories = predictions['categories']

# Print the number and names of categories in both files
print(f"Number of categories in ground truth: {len(gt_categories)}")
print("Ground truth categories:")
for cat in gt_categories:
    print(f"  - {cat['name']}")

print(f"\nNumber of categories in predicted file: {len(pred_categories)}")
print("Predicted file categories:")
for cat in pred_categories:
    print(f"  - {cat['name']}")

Number of categories in ground truth: 4
Ground truth categories:
  - paragraph
  - text_box
  - image
  - table

Number of categories in predicted file: 3
Predicted file categories:
  - text_box
  - paragraph
  - table


In [46]:
def extract_bounding_box_info(json_file, gt_or_pt):
    
    with open(json_file, 'r') as f:
        json_data = json.load(f)
        
    bbox_info = []
    if gt_or_pt == "gt":
        print("Passed argument is GROUND TRUTH ")
        for ann in json_data['annotations']:
            bbox = ann['bbox']
            bbox_info.append({
                'image_id': ann['image_id'],
                'bbox': [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]],
                'category_id': ann['category_id']
                })
        
        return bbox_info
    else:
        print("Passed argument is PREDICTIONS")
        for ann in json_data['annotations']:
            bbox = ann['bbox']
            bbox_info.append({
                'image_id': ann['image_id'],
                'bbox': [bbox[0], bbox[1], bbox[2], bbox[3]],
                'category_id': ann['category_id']
                })
        
        return bbox_info

In [47]:
ground_truth_bboxes = extract_bounding_box_info(gt_file_path, "gt")

Passed argument is GROUND TRUTH 


In [48]:
prediction_bboxes = extract_bounding_box_info(pred_file_path, "pt")

Passed argument is PREDICTIONS


In [ ]:
# print(ground_truth_bboxes)

In [50]:
print(prediction_bboxes)

[{'image_id': 0, 'bbox': [1154.0, 2437.0, 1275.0, 2454.0], 'category_id': 1}, {'image_id': 0, 'bbox': [306.0, 2437.0, 372.0, 2453.0], 'category_id': 1}, {'image_id': 0, 'bbox': [126.0, 2437.0, 193.0, 2463.0], 'category_id': 1}, {'image_id': 0, 'bbox': [1462.0, 2431.0, 1578.0, 2454.0], 'category_id': 1}, {'image_id': 0, 'bbox': [518.0, 2427.0, 661.0, 2464.0], 'category_id': 1}, {'image_id': 0, 'bbox': [458.0, 2427.0, 517.0, 2465.0], 'category_id': 1}, {'image_id': 0, 'bbox': [659.0, 2401.0, 725.0, 2417.0], 'category_id': 1}, {'image_id': 0, 'bbox': [582.0, 2401.0, 650.0, 2428.0], 'category_id': 1}, {'image_id': 0, 'bbox': [462.0, 2395.0, 573.0, 2416.0], 'category_id': 1}, {'image_id': 0, 'bbox': [1154.0, 2367.0, 1274.0, 2383.0], 'category_id': 1}, {'image_id': 0, 'bbox': [306.0, 2367.0, 376.0, 2383.0], 'category_id': 1}, {'image_id': 0, 'bbox': [129.0, 2367.0, 187.0, 2383.0], 'category_id': 1}, {'image_id': 0, 'bbox': [620.0, 2367.0, 682.0, 2383.0], 'category_id': 1}, {'image_id': 0, 'b

In [51]:
def calculate_iou(gt_bbox, pred_bbox):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Args:
        gt_bbox (list): Ground truth bounding box [xmin, ymin, xmax, ymax].
        pred_bbox (list): Prediction bounding box [xmin, ymin, xmax, ymax].

    Returns:
        float: Intersection over Union (IoU) score.
    """
    # Extract coordinates from ground truth and prediction bounding boxes
#     gt_x, gt_y, gt_w, gt_h = gt_bbox
    
    gt_xmin, gt_ymin, gt_xmax, gt_ymax = pred_bbox
    pred_xmin, pred_ymin, pred_xmax, pred_ymax = pred_bbox

    # Calculate coordinates of intersection rectangle
    x_left = max(gt_xmin, pred_xmin)
    y_top = max(gt_ymin, pred_ymin)
    x_right = min(gt_xmax, pred_xmax)
    y_bottom = min(gt_ymax, pred_ymax)

    # If there is no intersection, return 0
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # Calculate area of intersection
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate area of union
    gt_area = (gt_xmax - gt_xmin) * (gt_ymax - gt_ymin)
    pred_area = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
    union_area = gt_area + pred_area - intersection_area

    # Calculate IoU
    if union_area == 0:
        return 0.0
    iou = intersection_area / union_area

    return iou

In [53]:
def calculate_mAP(gt_bboxes, pred_bboxes, iou_threshold=0.5):
    """
    Calculates the average precision (AP) for a given IoU threshold.
    """
    num_classes = len(set([bbox['category_id'] for bbox in gt_bboxes]))
    print("num_classes", num_classes)
    ap_sum = 0
    for class_id in range(num_classes):
        class_gt_bboxes = [bbox for bbox in gt_bboxes if bbox['category_id'] == class_id]
        class_pred_bboxes = [bbox for bbox in pred_bboxes if bbox['category_id'] == class_id]
        if len(class_gt_bboxes) == 0 or len(class_pred_bboxes) == 0:
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!LENGTH 0")
            continue
        sorted_pairs = sorted([(i, j, calculate_iou(gt_bbox['bbox'], pred_bbox['bbox']))
                               for i, gt_bbox in enumerate(class_gt_bboxes)
                               for j, pred_bbox in enumerate(class_pred_bboxes)],
                              key=lambda x: x[2], reverse=True)
        precision = []
        recall = []
        used_pred_indices = set()
        for i, j, iou in sorted_pairs:
            if iou >= iou_threshold and j not in used_pred_indices:
                used_pred_indices.add(j)
                tp = len(used_pred_indices)
                fp = len(class_pred_bboxes) - tp
                fn = len(class_gt_bboxes) - tp
                precision.append(tp / (tp + fp))
                recall.append(tp / (tp + fn))
        for i in range(len(precision) - 2, -1, -1):
            precision[i] = max(precision[i], precision[i+1])
        ap = 0
        for i in range(len(precision)):
            if i == 0:
                delta_recall = recall[i]
            else:
                delta_recall = recall[i] - recall[i-1]
            ap += precision[i] * delta_recall
        ap_sum += ap
    mAP = ap_sum / num_classes
    return mAP

In [54]:
mAP = calculate_mAP(ground_truth_bboxes, prediction_bboxes, iou_threshold=0.5)
print(mAP)

num_classes 4
!!!!!!!!!!!!!!!!!!!!!!!!!!!LENGTH 0
0.0011056727394078455
